In [9]:
import pandas as pd
from sklearn.externals import joblib

from sklearn.base import BaseEstimator, TransformerMixin

In [10]:
class ItemSelector(BaseEstimator, TransformerMixin):
    # See http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html#example-hetero-feature-union-py

    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

class FillNaN(BaseEstimator, TransformerMixin):

    def __init__(self, replace=0):
        self.replace = replace

    def fit(self, x, y=None):
        return self

    def transform(self, serie):
        return serie.fillna(self.replace)

class Pass(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X


In [11]:
pipeline = joblib.load('pipeline.gz')
model = joblib.load('model.gz')

In [12]:
df_test = pd.read_csv("../test_final_100k.csv")
avisos = pd.read_pickle("avisos.pkl")
postulantes = pd.read_pickle("postulantes.pkl")
visitas_test = pd.read_pickle("visitas_test.pkl")

Hacemos merge con dataset de visitas. Podria suceder que hayan postulaciones sin visitas, como en el dataset de train, lo cual seria un error, pero asumimos que si hay un valor nulo en visita, es porque realmente no existio una visita de ese postulante para ese anuncio.

In [13]:
df_test = pd.merge(df_test, visitas_test, how="left", on=['idaviso', 'idpostulante'])
df_test.loc[df_test['visita_cantidad'].isnull(), 'visita_cantidad'] = 0

In [14]:
df_test = pd.merge(df_test, avisos, how='left', on='idaviso')
df_test = pd.merge(df_test, postulantes, how='left', on='idpostulante')

In [15]:
df_test.sample(5)

,id,idaviso,idpostulante,visita_cantidad,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,sexo,edad,educacion_nivel,educacion_estado,educacion
62677,62677,1112456289,X9lxR6K,4.0,Técnico en Seguridad,<p>Buscamos sumar a nuestro equipo de trabajo ...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Seguridad e Higiene,Benito Roggio ambiental,True,25.0,Universitario,En Curso,10.0
54541,54541,1112446707,1lbEBO,0.0,Vendedor en relacion a Inyeccion Diesel,"<p style="""">La empresa se dedica a la reparaci...",Capital Federal,Full-time,Otro,Ventas,Diagno Truck,True,53.0,Universitario,Graduado,11.0
81684,81684,1112466106,N5WQGJ,1.0,Asistente de Cuentas ( zona PALERMO - CABA ) U...,<p>En KaizenRH buscamos <strong>Asistente de C...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Kaizen Recursos Humanos,True,28.0,Universitario,En Curso,10.0
73257,73257,1112463356,KBrVLOq,2.0,Cajera/o,<p>Si soñás con pertenecer a una compañía insp...,Gran Buenos Aires,Part-time,Otro,Tesorería,PRÜNE,False,25.0,Universitario,En Curso,10.0
26112,26112,1112337868,5Y2Bqp,0.0,Secretaria para centro medico,<p>Importante Centro Medico esta solicitando u...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Secretaria,Estrategia Laboral S.A.,True,43.0,NaN,NaN,NaN


In [16]:
X_test = pipeline.transform(df_test)

In [17]:
y_pred = model.predict_proba(X_test)

In [18]:
df_test['sepostulo'] = y_pred[:, 1]

In [19]:
df_test[['id', 'sepostulo']].to_csv('submit04.csv', index=False)